In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
import numpy as np
from copy import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import pymedtermino

from metric_learn import NCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

from tpot import TPOTClassifier

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import sys
import getpass

username = getpass.getuser()
sys.path.append(f'/home/{username}/emr-scaner/')

from src.models.metric_learner import MetricLearningAlgorithm

In [24]:
df = pd.read_csv('../data/interim/meddra_data.csv')
meddra_labels = {v:k for k, v in enumerate(df['meddra'].unique())}
df['meddra_label'] = df['meddra'].apply(lambda x: int(meddra_labels[x]))
train, test = train_test_split(df, test_size=0.20)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(train['text'])

train['text_tokenized'] = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train['text']), maxlen=7).tolist()
test['text_tokenized'] = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test['text']), maxlen=7).tolist()
train['counts'] = train['meddra_label'].apply(lambda x: train[train['meddra_label']==x].shape[0])
#train = train[train['counts']>50]

X_train = np.array(train['text_tokenized'].to_list())
y_train = np.array(train['meddra_label'].to_list())

X_test = np.array(test['text_tokenized'].to_list())
y_test = np.array(test['meddra_label'].to_list())

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2153, 7), (2153,), (539, 7), (539,))

In [25]:

from sklearn.neighbors import KNeighborsClassifier
kneighb = KNeighborsClassifier(n_neighbors=73, p=1, weights='distance')
neigh.fit(X, y)


KNeighborsClassifier(n_neighbors=73, p=1, weights='distance')

In [ ]:
for name in tqdm(MetricLearningAlgorithm.available_mltools()):
    mltool = MetricLearningAlgorithm(mltool_name=name, 
                                    mltool_params={})
    
    kneighb = KNeighborsClassifier(n_neighbors=73, 
                                   p=1, 
                                   weights='distance')
    
    clf = make_pipeline(mltool, kneighb)
    try:
        clf.fit(X_train, y_train)
        print(name, clf.score(X_test, y_test))
    except Exception as e:
        print(name, e)

NCA 0.13358070500927643
LFDA 0.1261595547309833
LMNN not enough class labels for specified k (smallest class has 1)
MLKR 0.12987012987012986
RCA 0.13543599257884972


In [26]:
name, clf.score(X_test, y_test)

NotFittedError: This ITML_Supervised instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [27]:
kneighb.fit(X_train, y_train)
kneighb.score(X_test, y_test)

0.1836734693877551